In [1]:
import pandas as pd
import numpy as np
import lib
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import re as re
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba
import mpl_finance as mpf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectPercentile, chi2, SelectKBest
from sklearn.svm import SVC
from sklearn import model_selection, metrics, ensemble
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from matplotlib.font_manager import FontProperties
import time
import math
%matplotlib

ModuleNotFoundError: No module named 'lib'

### Read Files

In [ ]:
def stock_convert_dtype(stock_pd):
    stock_pd['年月日'] = pd.to_datetime(stock_pd['年月日'])
    stock_pd['開盤價(元)']= stock_pd['開盤價(元)'].apply(lambda x: x.replace(',', '')).astype('float')
    stock_pd['最高價(元)']= stock_pd['最高價(元)'].apply(lambda x: x.replace(',', '')).astype('float')
    stock_pd['最低價(元)']= stock_pd['最低價(元)'].apply(lambda x: x.replace(',', '')).astype('float')
    stock_pd['收盤價(元)']= stock_pd['收盤價(元)'].apply(lambda x: x.replace(',', '')).astype('float')
    stock_pd['成交量(千股)']= stock_pd['成交量(千股)'].apply(lambda x: x.replace(',', '')).astype('int')
    stock_pd['成交值(千元)']= stock_pd['成交值(千元)'].apply(lambda x: x.replace(',', '')).astype('int')
    stock_pd['成交筆數(筆)']= stock_pd['成交筆數(筆)'].apply(lambda x: x.replace(',', '')).astype('int')
    stock_pd['流通在外股數(千股)']= stock_pd['流通在外股數(千股)'].apply(lambda x: x.replace(',', '')).astype('int')
    stock_pd['本益比-TSE'] = stock_pd['本益比-TSE'].astype('str')
    stock_pd['本益比-TSE']= stock_pd['本益比-TSE'].apply(lambda x: x.replace(',', '')).astype('float')
    return stock_pd

In [ ]:
def get_file_hdr_idx(file_pd):
    file_idx2hdr = list(file_pd)
    file_hdr2idx = dict((hdr, idx) for idx, hdr in enumerate(file_idx2hdr))
    return file_idx2hdr, file_hdr2idx

In [ ]:
listed_2018_pd = pd.read_csv('bda2019_dataset/listed_company_2018.csv')
listed_2017_pd = pd.read_csv('bda2019_dataset/listed_company_2017.csv')
listed_2016_pd = pd.read_csv('bda2019_dataset/listed_company_2016.csv')
listed_2016_pd = stock_convert_dtype(listed_2016_pd)
listed_2017_pd = stock_convert_dtype(listed_2017_pd)
listed_2018_pd = stock_convert_dtype(listed_2018_pd)
stocks_pd = listed_2018_pd.append(listed_2017_pd, ignore_index=True)
stocks_pd = stocks_pd.append(listed_2016_pd, ignore_index=True)
stocks_idx2hdr, stocks_hdr2idx = get_file_hdr_idx(stocks_pd)
# stocks_pd.dtypes

In [ ]:
# counter_2018_pd = pd.read_csv('bda2019_dataset/counter_company_2018.csv')
# counter_2017_pd = pd.read_csv('bda2019_dataset/counter_company_2017.csv')
# counter_2016_pd = pd.read_csv('bda2019_dataset/counter_company_2016.csv')
# counter_2016_pd = stock_convert_dtype(listed_2016_pd)
# counter_2017_pd = stock_convert_dtype(listed_2017_pd)
# counter_2018_pd = stock_convert_dtype(listed_2018_pd)
# stocks_pd = stocks_pd.append(counter_2018_pd, ignore_index=True)
# stocks_pd = stocks_pd.append(counter_2017_pd, ignore_index=True)
# stocks_pd = stocks_pd.append(counter_2016_pd, ignore_index=True)
# stocks_idx2hdr, stocks_hdr2idx = get_file_hdr_idx(stocks_pd)

In [ ]:
news_pd = pd.read_csv('bda2019_dataset/news.csv',encoding='big5')
news_pd['post_time'] = pd.to_datetime(news_pd['post_time'])
news_pd['content'] = news_pd['content'].astype('str')
news_idx2hdr, news_hdr2idx = get_file_hdr_idx(news_pd)
# news_pd.dtypes

In [ ]:
bbs_pd = pd.read_csv('bda2019_dataset/bbs-utf8.csv')
bbs_pd['post_time'] = pd.to_datetime(bbs_pd['post_time'])
bbs_pd['content'] = bbs_pd['content'].astype('str')
bbs_idx2hdr, bbs_hdr2idx = get_file_hdr_idx(bbs_pd)
# bbs_pd.dtypes

In [ ]:
forum_pd = pd.read_csv('bda2019_dataset/forum-utf8.csv')
forum_pd['post_time'] = pd.to_datetime(forum_pd['post_time'])
forum_pd['content'] = forum_pd['content'].astype('str')
forum_idx2hdr, forum_hdr2idx = get_file_hdr_idx(forum_pd)
# forum_pd.dtypes

In [ ]:
weight_pd = pd.read_csv('權值股.csv')
weight_idx2hdr, weight_hdr2idx = get_file_hdr_idx(weight_pd)

### Converts to array

In [ ]:
stocks_np = stocks_pd.values
stocks_np = stocks_np[::-1] # reverse
news_np = news_pd.values
bbs_np = bbs_pd.values
forum_np = forum_pd.values
weight_np = weight_pd.values

### Functions

In [ ]:
def get_companys(company, stock_np, name_index=0):
    indexs = []
    for i in range(stock_np.shape[0]):
        if company in stock_np[i][name_index]:
            indexs.append(i)
    new_np = stock_np[indexs,:]
#     print('Company Shape = ', new_np.shape)
    return new_np

In [ ]:
def get_by_all_keyword(keyword_list, org_array_np, from_index=0):
    indexs = []
    for i in range(org_array_np.shape[0]):
        if all(keyword in org_array_np[i][from_index] for keyword in keyword_list):
            indexs.append(i)
    new_array_np = org_array_np[indexs,:]
#     print('New array Shape = ', new_ar?ray_np.shape)
    return new_array_np

In [ ]:
def get_by_any_keyword(keyword_list, org_array_np, from_index=0):
    indexs = []
    for i in range(org_array_np.shape[0]):
        if any(keyword in org_array_np[i][from_index] for keyword in keyword_list):
            indexs.append(i)
    new_array_np = org_array_np[indexs,:]
#     print('New array Shape = ', new_array_np.shape)
    return new_array_np

In [ ]:
def get_ma(price, avg_num):
    ones = [1] * avg_num
    ma = np.convolve(price, ones)
    ma = ma[0:len(price)]/avg_num
    return ma

In [ ]:
def get_idx_by_2line(line1, line2):
    line_diff = line1-line2
    line1_greater = np.where(line_diff>0, 1, 0)
    line1_greater_org = np.insert(line1_greater, len(line1_greater)-1, 0)
    line1_greater_dly = np.insert(line1_greater, 0, 0)
    line1_lead = line1_greater_org & ~line1_greater_dly
    line2_lead = ~line1_greater_org & line1_greater_dly
    line1_lead_idx = np.where(line1_lead>0)[0]
    line2_lead_idx = np.where(line2_lead>0)[0]
    return line1_lead_idx, line2_lead_idx

In [ ]:
def get_valid_idx(up_idx, down_idx, up_thr, down_thr, stock_np, close_price_idx):
    up_idx = up_idx[(up_idx>3) & (up_idx<len(company_stock_np)-3)]
    down_idx = down_idx[(down_idx>3) & (down_idx<len(company_stock_np)-3)]
    post1 = company_stock_np[up_idx+1,close_price_idx]
    post2 = company_stock_np[up_idx+2,close_price_idx]
    post3 = company_stock_np[up_idx+3,close_price_idx]
    pre1 = company_stock_np[up_idx-1,close_price_idx]
    pre2 = company_stock_np[up_idx-2,close_price_idx]
    pre3 = company_stock_np[up_idx-3,close_price_idx]
    up_amp = (post1+post2+post3)/(pre1+pre2+pre3)
    up_vld_idx = up_idx[up_amp>up_thr]
    post1 = company_stock_np[down_idx+1,close_price_idx]
    post2 = company_stock_np[down_idx+2,close_price_idx]
    post3 = company_stock_np[down_idx+3,close_price_idx]
    pre1 = company_stock_np[down_idx-1,close_price_idx]
    pre2 = company_stock_np[down_idx-2,close_price_idx]
    pre3 = company_stock_np[down_idx-3,close_price_idx]
    down_amp = (pre1+pre2+pre3)/(post1+post2+post3)
    down_vld_idx = down_idx[down_amp>down_thr]
    return up_vld_idx, down_vld_idx, up_amp, down_amp

In [ ]:
def get_file_hdr_idx(file_pd):
    file_idx2hdr = list(file_pd)
    file_hdr2idx = dict((hdr, idx) for idx, hdr in enumerate(file_idx2hdr))
    return file_idx2hdr, file_hdr2idx

In [ ]:
def get_dates_by_idx(idx, stock_np, date_idx, pre_date):
    try:
        dates = stock_np[idx-pre_date, date_idx]
    except:
        dates = []
    return dates

In [ ]:
def get_ma_up_dn_dates(company_stock_np,stocks_hdr2idx,thr_up,thr_lo,pre_day):
    close_price = company_stock_np[:,stocks_hdr2idx['收盤價(元)']]
    ma20 = get_ma(close_price, 20)
    ma5 = get_ma(close_price, 5)
    up_idx, dn_idx = get_idx_by_2line(ma5, ma20)
    up_vld_idx, dn_vld_idx, up_amp, dn_amp = get_valid_idx(
        up_idx, dn_idx, thr_up, thr_lo, company_stock_np, stocks_hdr2idx['收盤價(元)'])
    up_dates = get_dates_by_idx(up_vld_idx, company_stock_np, stocks_hdr2idx['年月日'], pre_day)
    dn_dates = get_dates_by_idx(dn_vld_idx, company_stock_np, stocks_hdr2idx['年月日'], pre_day)
    return up_dates, dn_dates, ma20, ma5

In [ ]:
def get_kd(close_price):
    # Lowest price in past 9 days
    low_price_matrix = np.ones(shape = (9,len(low_price)+8))*10000
    for n in range(0,9):
        low_price_matrix[n,n:len(low_price)+n] = low_price
    low_price_pass9 = np.amin(low_price_matrix, axis=0)
    low_price_pass9 = low_price_pass9[0:len(low_price)]
    # Highest price in past 9 days
    high_price_matrix = np.zeros(shape = (9,len(high_price)+8))
    for n in range(0,9):
        high_price_matrix[n,n:len(high_price)+n] = high_price
    high_price_pass9 = np.amax(high_price_matrix, axis=0)
    high_price_pass9 = high_price_pass9[0:len(high_price)]
    # rsv
    sp_idx = np.where(high_price_pass9==low_price_pass9) # special case
    low_price_pass9[sp_idx] = low_price_pass9[sp_idx]-1
    rsv = (close_price-low_price_pass9)/(high_price_pass9-low_price_pass9)
    # KD
    kd_k = 1/3*rsv
    kd_d = 1/3*rsv
    for n in range(1,len(rsv)):
        kd_k[n] = 2/3*kd_k[n-1] + 1/3*rsv[n]
        kd_d[n] = 2/3*kd_d[n-1] + 1/3*kd_k[n]
    return kd_k, kd_d

In [ ]:
def get_kd_up_dn_dates(company_stock_np,stocks_hdr2idx,thr,pre_day):
    close_price = company_stock_np[:,stocks_hdr2idx['收盤價(元)']]
    kd_k, kd_d = get_kd(close_price)
    up_idx, dn_idx = get_idx_by_2line(kd_k, kd_d)
    up_vld_idx, dn_vld_idx, up_amp, dn_amp = get_valid_idx(
        up_idx, dn_idx, thr, thr, company_stock_np, stocks_hdr2idx['收盤價(元)'])
    up_dates = get_dates_by_idx(up_vld_idx, company_stock_np, stocks_hdr2idx['年月日'], pre_day)
    dn_dates = get_dates_by_idx(dn_vld_idx, company_stock_np, stocks_hdr2idx['年月日'], pre_day)
    return up_dates, dn_dates, kd_k, kd_d

In [ ]:
def get_vol_up_dn_dates(company_stock_np,stocks_hdr2idx,ranks_per_year,pre_day):
    close_price = company_stock_np[:,stocks_hdr2idx['收盤價(元)']]
    volumn = company_stock_np[:,stocks_hdr2idx['成交筆數(筆)']]
    open_price = company_stock_np[:,stocks_hdr2idx['開盤價(元)']]
    close_price = company_stock_np[:,stocks_hdr2idx['收盤價(元)']]
    up_idx = []
    dn_idx = []
    for year in range(2016,2019):
        year_idx = np.where( (company_stock_np[:,stocks_hdr2idx['年月日']]>=datetime(year  , 1, 1, 0, 0))
                           & (company_stock_np[:,stocks_hdr2idx['年月日']]< datetime(year+1, 1, 1, 0, 0))
                           )
        try:
            init_idx = year_idx[0][0]
        except:
            init_idx = 0
        top_vol_idx = np.argsort(volumn[year_idx])[::-1][0:ranks_per_year]+init_idx
        dn_vol = open_price[top_vol_idx]>close_price[top_vol_idx]
        for n in range(0,len(dn_vol)):
            if dn_vol[n]:
                dn_idx.append(top_vol_idx[n])
            else:
                up_idx.append(top_vol_idx[n])
    up_dates = get_dates_by_idx(np.array(up_idx), company_stock_np, stocks_hdr2idx['年月日'], pre_day)
    dn_dates = get_dates_by_idx(np.array(dn_idx), company_stock_np, stocks_hdr2idx['年月日'], pre_day)
    return up_dates, dn_dates

In [ ]:
def get_doc_by_date(dates, data, hdr2idx):
    docs = []
    date_index = hdr2idx['post_time']
    title_index = hdr2idx['title']
    content_index = hdr2idx['content']
    author_indx = hdr2idx['author']
    for row in data:
        for date in dates:
            if row[date_index].date() == date.date():
                temp_doc = '%s %s %s'%(row[title_index],row[title_index],row[content_index])
                docs.append(remove_notation(temp_doc))
    return docs

In [ ]:
def get_doc_cnt_by_date(dates, data, hdr2idx):
    doc_cnt = []
    date_index = hdr2idx['post_time']
    title_index = hdr2idx['title']
    content_index = hdr2idx['content']
    author_indx = hdr2idx['author']
    for date in dates:
        doc_cnt_tmp = 0
        for row in data:
            if row[date_index].date() == date.date():
                doc_cnt_tmp += 1
        doc_cnt.append(doc_cnt_tmp)
    return doc_cnt

In [ ]:
def remove_notation(text):
    text = text.lower()
    text = re.sub('<br>',' ',text)
    text = re.sub('http[\a-z0-9]* ',' ',text)
    text = re.sub('\d+',' ',text) # remove number
    text = re.sub('[^\w\s]',' ',text) # remove punctuation
    text = re.sub(r'\b\w\b',' ',text) # remove single eng character
    text = re.sub(r'[\u4e00-\u9fa5][a-z]\b','',text)
    text = re.sub(r'\b[a-z][\u4e00-\u9fa5][a-z]','',text)
    text = re.sub('\s+',' ',text) # combine space
    return text

In [ ]:
def get_ctgry_doc_by_date(company, ctgry_np, ctgry_hdr2idx, up_dates, dn_dates):
    ctgry = ctgry_np[0,1]
    company_np = get_companys(company=company, stock_np=ctgry_np, name_index=ctgry_hdr2idx['content'])
    up_docs = np.array(get_doc_by_date(list(up_dates), company_np, ctgry_hdr2idx) )
    dn_docs = np.array(get_doc_by_date(list(dn_dates), company_np, ctgry_hdr2idx) )
    return up_docs, dn_docs

In [ ]:
def get_ctgry_doc_by_allkeyword_date(keyword_list, ctgry_np, ctgry_hdr2idx, up_dates, dn_dates):
    ctgry = ctgry_np[0,1]
    kw_np = get_by_all_keyword(keyword_list=keyword_list, org_array_np=ctgry_np, from_index=ctgry_hdr2idx['content'])
    up_docs = np.array(get_doc_by_date(list(up_dates), kw_np, ctgry_hdr2idx) )
    dn_docs = np.array(get_doc_by_date(list(dn_dates), kw_np,ctgry_hdr2idx) )
    up_date_doc_cnt = get_doc_cnt_by_date(list(up_dates), ctgry_np, ctgry_hdr2idx)
    dn_date_doc_cnt = get_doc_cnt_by_date(list(dn_dates), ctgry_np, ctgry_hdr2idx)

    print('{}: There are {} up docs and {} down {} docs'.format(company, len(up_docs), len(dn_docs), ctgry.upper()))
    return up_docs, dn_docs, up_date_doc_cnt, dn_date_doc_cnt

In [ ]:
def get_ctgry_doc_by_anykeyword_date(keyword_list, ctgry_np, ctgry_hdr2idx, up_dates, dn_dates):
    ctgry = ctgry_np[0,1]
    
    kw_np = get_by_any_keyword(keyword_list, ctgry_np, ctgry_hdr2idx['content'])
    print(len(kw_np), len(ctgry_np))
    print(keyword_list)
    up_docs = np.array(get_doc_by_date(list(up_dates), kw_np, ctgry_hdr2idx) )
    dn_docs = np.array(get_doc_by_date(list(dn_dates), kw_np, ctgry_hdr2idx) )

    print('{}: There are {} up docs and {} down {} docs'.format(company, len(up_docs), len(dn_docs), ctgry.upper()))
    return up_docs, dn_docs

In [ ]:
def get_stopwords(file_name):
    stopwords = []
    with open(file_name,'r', encoding='UTF-8') as f1:
        for line in f1:
            stopwords.append(line.strip())
    stopwords.append('nan')
    stopwords.append(' ')
    return stopwords

In [ ]:
def get_count_tool(corpus):
    vectorizer = CountVectorizer(tokenizer=jieba.cut, analyzer='word', min_df=2, stop_words=stopwords)
    X = vectorizer.fit_transform(corpus)
#     print(X.shape)
    return X, vectorizer

In [ ]:
def my_chi2(all_vectors, clas_vectors):
    expected =all_vectors.sum(axis=0)*len(clas_vectors)/len(all_vectors)
    clas_value = clas_vectors.sum(axis=0)
    sign_idx = np.where(clas_value-expected < 0)[0]
    chi2_value = np.power(clas_value-expected,2)/expected
    chi2_value[sign_idx] = chi2_value[sign_idx] * -1
    return chi2_value

In [ ]:
def get_df(term_list, doc_list):
    df_list = list()
    for keyword in term_list:
        df = 0
        for text in doc_list:
            if keyword in text:
                df = df + 1
    return df_list

In [ ]:
def get_tf(term_list, doc_list):
    tf_list = list()
    for text in doc_list:
        tf_tmp = list()
        for keyword in term_list:
            tf_tmp.append(text.count(keyword)) # append tf for 1 doc
        tf_list.append(tf_tmp) # append tf of doc to tf_list
    tf_np = np.array(tf_list)
    return tf_np

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Main

### Doc Selection (News)

In [ ]:
# keyword_list = ['刪除','請問','請教','自刪','繼承','信用卡','匯率','保單','遊戲','關閉']
keyword_list = ['盤後','盤中','盤前','交易概況','上市認購','晨訊','各報要聞','報價簡訊','台北股市','海外存託憑證',
                '國內匯市','證交所','y早報','y晚報','焦點新聞','投顧','晨間解析','集中市場']
keyword_list_a = ['買進','賣出']
keyword_list_b = ['上漲','下跌']
keyword_list_c = ['跌破','衝上']
keyword_list_d = ['買超','賣超']
# ctgry_np = forum_np
# ctgry_hdr2idx = forum_hdr2idx
ctgry_np = news_np
ctgry_hdr2idx = news_hdr2idx

indexs = []
ctgry_trunc_np = []
for i in range(ctgry_np.shape[0]):
    context = (ctgry_np[i,ctgry_hdr2idx['title']] + ctgry_np[i,ctgry_hdr2idx['content']]).lower()

    if not (any(keyword in context for keyword in keyword_list) or
        all(keyword_a in context for keyword_a in keyword_list_a) or
        all(keyword_b in context for keyword_b in keyword_list_b) or 
        all(keyword_c in context for keyword_c in keyword_list_c) or
        all(keyword_d in context for keyword_d in keyword_list_d) ):
        indexs.append(i)
ctgry_trunc_np = ctgry_np[indexs,:]

print('Original {} docs, reduced to {} docs'.format(len(ctgry_np), len(ctgry_trunc_np)))

### Paramter List (Training sets)

In [ ]:
# Parameters
pre_day_t = 2
pre_day_dn = pre_day_t
test_cut_time = datetime(2018, 1, 1, 0, 0)
up_tick = 1
dn_tick = 0
no_tick = 2
sigma = 0.0
sigma_dn = 0.0
company_list= ['台積電']
# company_list= ['中信金']
# company_list= ['第一金']
# company_list = ['台積電', '鴻海', '大立光', '南亞', '台塑', '華新', '國泰金', '聯發科']
# company_list = ['台積電']
# company_list = ['鴻海']
# company_list = ['國泰金']
# company_list = ['大立光']

### Paramter List (Testing sets)

In [ ]:
# company = '國泰金'
# company = '台積電'
# test_cut_time = datetime(2018,7,1, 0, 0)
pre_day = 3
tcnt_ma_f_len = 5
tcnt_ma_s_len = 10

### Training

### Find Up & Down docs

In [ ]:
print (time.asctime( time.localtime(time.time()) ))
up_docs_all = []
dn_docs_all = []
# ctgry_np = ctgry_trunc_np
# ctgry_hdr2_idx = forum_hdr2idx
# ctgry_np = news_np
# ctgry_hdr2idx = news_hdr2idx
for company in company_list:
    # Grep company stocks
    company_stock_np = get_companys(company, stocks_np, stocks_hdr2idx['證券代碼'])
    # Cut test set for future use
    company_stock_train_np = company_stock_np[company_stock_np[:,1]<test_cut_time,:]
    pe_ratio = company_stock_train_np[:,stocks_hdr2idx['本益比-TSE']]
    pb_ratio = company_stock_train_np[:,stocks_hdr2idx['股價淨值比-TSE']]
    open_price = company_stock_train_np[:,stocks_hdr2idx['開盤價(元)']]
    high_price = company_stock_train_np[:,stocks_hdr2idx['最高價(元)']]
    low_price = company_stock_train_np[:,stocks_hdr2idx['最低價(元)']]
    close_price = company_stock_train_np[:,stocks_hdr2idx['收盤價(元)']]
    volumn = company_stock_train_np[:,stocks_hdr2idx['成交筆數(筆)']]
    
    # up down ticks
    close_price_org = np.insert(close_price, len(close_price)-1, close_price[-1])
    close_price_dly = np.insert(close_price, 0, close_price[0])
    for n in range(1,pre_day_t):
        close_price_org = np.insert(close_price_org, len(close_price_org)-1, close_price_org[-1])
        close_price_dly = np.insert(close_price_dly, n, close_price[n])
    close_price_percent = (close_price_org-close_price_dly) / close_price_org
    up_down = np.ones(len(close_price_org))*no_tick
    up_down[np.where(close_price_percent> sigma)[0]] = up_tick
    up_down[np.where(close_price_percent<-sigma_dn)[0]] = dn_tick

    # MA, KD, Volumn
#     ma_up_dates, ma_dn_dates, ma20, ma5 = get_ma_up_dn_dates(company_stock_train_np,stocks_hdr2idx,1.01,1.01,pre_day_t)
#     kd_up_dates, kd_dn_dates, kd_k, kd_d = get_kd_up_dn_dates(company_stock_train_np,stocks_hdr2idx,1.05,pre_day_t)
#     vol_up_dates, vol_dn_dates = get_vol_up_dn_dates(company_stock_train_np,stocks_hdr2idx,5,pre_day_t)
#     up_dates = ma_up_dates
#     dn_dates = ma_dn_dates
    # up down dates
    up_dates = (company_stock_np[np.where(up_down==up_tick),stocks_hdr2idx['年月日']] - timedelta(days=pre_day_t))[0]
    dn_dates = (company_stock_np[np.where(up_down==dn_tick),stocks_hdr2idx['年月日']] - timedelta(days=pre_day_dn))[0]
    
    up_docs, dn_docs = get_ctgry_doc_by_date(company, ctgry_np, ctgry_hdr2idx, up_dates, dn_dates)

    up_docs_all = np.concatenate((up_docs_all, up_docs))
    dn_docs_all = np.concatenate((dn_docs_all, dn_docs))
print('There are {} up docs, {} down docs'.format(len(up_docs_all), len(dn_docs_all)))
print (time.asctime( time.localtime(time.time()) ))

In [ ]:
up_docs_all_new = up_docs_all
dn_docs_all_new = dn_docs_all

## Feature Selection

### Manual chi2

In [ ]:
feature_num = 10000

In [ ]:
stopwords = get_stopwords('stopwords.txt')
up_all_docs = up_docs_all_new
dn_all_docs = dn_docs_all_new
# up_all_docs = up_docs_all
# dn_all_docs = dn_docs_all
all_docs = np.append(up_all_docs  ,dn_all_docs)
chi_all_x, chi_all_vec = get_count_tool(all_docs)
y = np.append([up_tick]*len(up_all_docs),[dn_tick]*len(dn_all_docs))

In [ ]:
len(all_docs)

In [ ]:
chi_all_x.shape

In [ ]:
chi_all_matrix = chi_all_x.toarray()
chi_up_matrix = chi_all_matrix[0:len(up_all_docs)]
chi_down_matrix = chi_all_matrix[len(up_all_docs):len(chi_all_matrix)]

In [ ]:
chi2_up = my_chi2(chi_all_matrix, chi_up_matrix)
chi2_down = my_chi2(chi_all_matrix, chi_down_matrix)

In [ ]:
max_indexs = np.argsort(chi2_up)[::-1]
tokens = np.array(chi_all_vec.get_feature_names())
chi2_up_features = tokens[max_indexs[0:feature_num]]
# chi2_up_features = tokens[max_indexs]

In [ ]:
chi2_up_x = chi_all_matrix[0:len(up_all_docs),max_indexs[0:feature_num]]
# chi2_up_x = chi_all_matrix[0:len(up_all_docs),max_indexs]

In [ ]:
chi2_up_features[0:100]

In [ ]:
max_indexs = np.argsort(chi2_down)[::-1]
tokens = np.array(chi_all_vec.get_feature_names())
chi2_down_features = tokens[max_indexs]

In [ ]:
chi2_dn_x = chi_all_matrix[len(up_all_docs):len(chi_all_matrix),max_indexs[0:feature_num]]
# chi2_dn_x = chi_all_matrix[len(up_all_docs):len(chi_all_matrix),max_indexs]

In [ ]:
chi2_down_features[0:100]

In [ ]:
chi2_f_row = len(up_all_docs)+len(dn_all_docs)
chi2_up_fs = list(chi2_up_features[0:feature_num]).copy()
chi2_dn_fs = list(chi2_down_features[0:feature_num]).copy()
# chi2_up_fs = list(chi2_up_features).copy()
# chi2_dn_fs = list(chi2_down_features).copy()
chi2_up_fs=[i for i in chi2_up_fs if i not in chi2_dn_fs]
chi2_dn_fs=[i for i in chi2_dn_fs if i not in chi2_up_fs]
chi2_fs = chi2_up_fs + chi2_dn_fs
chi2_f_col = len(chi2_up_fs) + len(chi2_dn_fs)
chi2_features = np.zeros((chi2_f_row, chi2_f_col))
chi2_features[0:len(up_all_docs),0:len(chi2_up_fs)] = chi2_up_x[:,0:len(chi2_up_fs)]
chi2_features[len(up_all_docs):chi2_f_row,len(chi2_up_fs):chi2_f_col] = chi2_dn_x[:,0:len(chi2_dn_fs)]

In [ ]:
trans_x = chi2_features

In [ ]:
y = np.append([up_tick]*len(up_all_docs),[dn_tick]*len(dn_all_docs))

In [ ]:
all_kw = chi2_fs
# print(len(all_kw))
print('There are {} features'.format(len(all_kw)))

### Training model

In [ ]:
kf = StratifiedKFold(n_splits=3, random_state=0, shuffle=False)
print(kf)
forest = ensemble.RandomForestClassifier(n_estimators = 100)
# my_svm = SVC(gamma='auto')
for train_index, test_index in kf.split(trans_x,y):
    X_train, X_test = trans_x[train_index], trans_x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    forest_fit = forest.fit(X_train, y_train)
    test_y_predicted = forest.predict(X_test)
#     svc_fit = my_svm.fit(X_train, y_train)
#     test_y_predicted = my_svm.predict(X_test)
#     test_y_predicted = np.logical_not(test_y_predicted).astype(int)
    target_name = ['Down', 'Up']
    print(metrics.classification_report(y_test, test_y_predicted, target_names=target_name))
    accuracy = metrics.accuracy_score(y_test, test_y_predicted)
    print('Accuracy = ',accuracy)

### Testing

In [ ]:
company_stock_np = get_companys(company=company, stock_np=stocks_np, name_index=stocks_hdr2idx['證券代碼'])
company_stock_test_np = company_stock_np[company_stock_np[:,1]>=test_cut_time,:]
pe_ratio = company_stock_test_np[:,stocks_hdr2idx['本益比-TSE']]
pb_ratio = company_stock_test_np[:,stocks_hdr2idx['股價淨值比-TSE']]
open_price = company_stock_test_np[:,stocks_hdr2idx['開盤價(元)']]
high_price = company_stock_test_np[:,stocks_hdr2idx['最高價(元)']]
low_price = company_stock_test_np[:,stocks_hdr2idx['最低價(元)']]
close_price = company_stock_test_np[:,stocks_hdr2idx['收盤價(元)']]
volumn = company_stock_test_np[:,stocks_hdr2idx['成交筆數(筆)']]

close_price_org = np.insert(close_price, len(close_price)-1, close_price[-1])
close_price_dly = np.insert(close_price, 0, close_price[0])
for n in range(1,pre_day):
    close_price_org = np.insert(close_price_org, len(close_price_org)-1, close_price_org[-1])
    close_price_dly = np.insert(close_price_dly, n, close_price[n])

up_down = np.ones(len(close_price_org))*no_tick
up_down = np.where(close_price_org>close_price_dly,up_tick,up_down)
up_down = np.where(close_price_org<close_price_dly,dn_tick,up_down)

company_ctgry_np = get_companys(company, ctgry_trunc_np, ctgry_hdr2idx['content'])
# company_ctgry_np = get_companys(company, news_np, news_hdr2idx['content'])
# company_forum_np = get_by_any_keyword(keyword_list, forum_np, forum_hdr2idx['content'])
# ctgry_hdr2idx = forum_hdr2idx
predict_y = []
actual_y = []
ttl_doc_cnt = []
up_doc_cnt = []
dn_doc_cnt = []
open_price_array = []
close_price_array = []
high_price_array = []
low_price_array = []
test_day_array = []
p_up_down = 0 # 1st guess
ttl_doc_ma_f = list(np.zeros(tcnt_ma_f_len))
ttl_doc_ma_s = list(np.zeros(tcnt_ma_s_len))
ttl_ma_cross = 1
for test_day in range(0,len(company_stock_test_np)-10):

    today_date = company_stock_test_np[test_day:test_day+1,stocks_hdr2idx['年月日']]
    
    future_idx = test_day+pre_day
    open_price_array.append(list(company_stock_test_np[future_idx:future_idx+1,stocks_hdr2idx['開盤價(元)']])[0])
    close_price_array.append(list(company_stock_test_np[future_idx:future_idx+1,stocks_hdr2idx['收盤價(元)']])[0])
    high_price_array.append(list(company_stock_test_np[future_idx:future_idx+1,stocks_hdr2idx['最高價(元)']])[0])
    low_price_array.append(list(company_stock_test_np[future_idx:future_idx+1,stocks_hdr2idx['最低價(元)']])[0])

    test_day_array.append(today_date)
    actual_up_dn = up_down[future_idx]
    company_doc = np.array(get_doc_by_date(list(np.array(today_date)), company_ctgry_np, ctgry_hdr2idx))

    if (company_doc.size>2):
        chi_all_x, chi_all_vec = get_count_tool(company_doc)
        tokens = np.array(chi_all_vec.get_feature_names())
        company_tf_np = np.zeros((company_doc.size,len(all_kw)))
        for n, kw in enumerate(all_kw):
            try:
                idx = np.where(tokens==kw)[0][0]
            except:
                idx = -1

            if (idx>=0):
                company_tf_np[:,n] = chi_all_x.toarray()[:,idx]

        fpredict = forest.predict(company_tf_np)
        unique, counts = np.unique(fpredict, return_counts=True)
        if unique.size==1:
            p_up_down = unique[0]
        else:
            p_up_down = unique[1] if counts[1]>counts[0] else unique[0]
            
    ttl_doc_cnt.append(company_doc.size)
    ttl_doc_ma_f.append(company_doc.size)
    ttl_doc_ma_f.pop(0)
    ttl_doc_ma_s.append(company_doc.size)
    ttl_doc_ma_s.pop(0)
    ttl_ma_f = np.mean(ttl_doc_ma_f)
    ttl_ma_s = np.mean(ttl_doc_ma_s)
    ttl_ma_cross_dly = ttl_ma_cross
    if ttl_ma_f<ttl_ma_s:
        ttl_ma_cross = 0
    else:
        ttl_ma_cross = 1
    ttl_ma_cross_edge = ttl_ma_cross==1 and ttl_ma_cross_dly==0
    
    if company_doc.size<=2:
        up_doc_cnt.append(0)
        dn_doc_cnt.append(0)   
    elif unique.size==1:
        if unique[0]==dn_tick:
            up_doc_cnt.append(0)
            dn_doc_cnt.append(counts[0])
        else:
            up_doc_cnt.append(counts[0])
            dn_doc_cnt.append(0)
    else:
        up_doc_cnt.append(counts[up_tick])
        dn_doc_cnt.append(counts[dn_tick])
    
    if p_up_down == up_tick and up_doc_cnt[-1]>0:
        portion = (up_doc_cnt[-1]-dn_doc_cnt[-1])/up_doc_cnt[-1]
    elif p_up_down == dn_tick  and dn_doc_cnt[-1]>0:
        portion = (dn_doc_cnt[-1]-up_doc_cnt[-1])/dn_doc_cnt[-1]
    else:
        portion = -1
    if company_doc.size>5 and ttl_ma_cross_edge and ttl_ma_f-ttl_ma_s>0.04 and actual_up_dn!=no_tick:
        print('Test Date= {}, DocCnt={}(U {} D {} P {:.2f}), Prediction={}, Acutual={}'.
              format(today_date+timedelta(days=pre_day), company_doc.size, up_doc_cnt[-1], dn_doc_cnt[-1], portion,
                     p_up_down, actual_up_dn) )
        predict_y.append(p_up_down)
        actual_y.append(actual_up_dn)
ma20 = get_ma(close_price_array, 20)
ma5 = get_ma(close_price_array, 5)
accuracy = metrics.accuracy_score(actual_y, predict_y)
precision = metrics.precision_score(actual_y, predict_y, average='micro')
recall = metrics.recall_score(actual_y, predict_y, average='micro')
f1 = metrics.f1_score(actual_y, predict_y, average='weighted')
target_name = ['DOWN', 'UP']
print(metrics.classification_report(actual_y, predict_y, target_names=target_name))
print('{} Tested accuracy = {}'.format(company, accuracy))
ttl_doc_cnt_ma1 = get_ma(ttl_doc_cnt, 5)
ttl_doc_cnt_ma2 = get_ma(ttl_doc_cnt, 10)

In [ ]:
fig = plt.figure()
font_set = FontProperties(fname=r"c:\windows\fonts\kaiu.ttf", size=20)
plt.title('DocCnts-'+company, fontproperties=font_set)
ax = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)
ax2.plot(ttl_doc_cnt,color='olive', linewidth=4,  marker="s", label='Total Docs')
ax2.plot(up_doc_cnt,color='blue', linewidth=1,  marker="s", label='Up Docs')
ax2.plot(dn_doc_cnt,color='red', linewidth=1,  marker="s", label='Down Docs')
ax3.plot(np.concatenate(([np.NaN]*5,ttl_doc_cnt_ma1[5:len(ttl_doc_cnt_ma1)])),color='purple', linewidth=3, label='Docs MA_F')
ax3.plot(np.concatenate(([np.NaN]*10,ttl_doc_cnt_ma2[10:len(ttl_doc_cnt_ma2)])),color='orchid', linewidth=3, label='Docs MA_S')
ax.plot(np.concatenate(([np.NaN]*20,ma20[20:len(ma20)])),color='magenta', linewidth=2, label='MA20')
ax.plot(np.concatenate(([np.NaN]*5,ma5[5:len(ma5)])),color='green', linewidth=2, label='MA5')
ax.legend()
ax2.legend()
ax3.legend()
new_date = []
for dt in test_day_array:
    new_date.append(dt[0].strftime("%Y/%m/%d"))    
ax.set_xticks(range(0, len(new_date), 10))
ax.set_xticklabels(new_date[::10], rotation=30)
ax3.set_xticks(range(0, len(new_date), 10))
ax3.set_xticklabels(new_date[::10], rotation=30)
mpf.candlestick2_ochl(ax, open_price_array, close_price_array, high_price_array, low_price_array,
                     width=0.6, colorup='r', colordown='g', alpha=0.75); 